In [ ]:
from flexx import flx
from flexx import ui
import os
import pickle
import numpy as np
import pandas as pd
import cv2
import os
import csv
from io import BytesIO
import base64
import urllib
import matplotlib.pyplot as plt
import skimage
import skimage.segmentation as seg
import skimage.transform as tran
from skimage import io,transform,filters, color, measure, segmentation, morphology, feature
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
import matplotlib.patches as mpatches
import math
from sklearn.externals import joblib

In [ ]:
from flexx import flx
from flexx import ui
import os
import pickle
import numpy as np
import pandas as pd
import cv2
import os
import csv
from io import BytesIO
import base64
import urllib
import matplotlib.pyplot as plt
import skimage
import skimage.segmentation as seg
import skimage.transform as tran
from skimage import io,transform,filters, color, measure, segmentation, morphology, feature
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
import matplotlib.patches as mpatches
import math
from sklearn.externals import joblib

In [ ]:
X_dic = {'Image name':[],'Image type':[],'Image size':[],'Label':[],'Annotation':[]}
for img in X:
    i = io.imread(img)
    #print(eth_label[str(img[8])])
    X_dic['Image name'].append(img[10:])
    X_dic['Image type'].append(img[-3:])
    X_dic['Image size'].append(i.shape)
    X_dic['Label'].append(img[8])
    X_dic['Annotation'].append(eth_label[str(img[8])])

In [ ]:
#Feature read
LBP = open('LBP.pickle','rb')
X_lbp = pickle.load(LBP)
#print(X_lbp)

HIST = open('HIST.pickle','rb')
X_hist = pickle.load(HIST)
#print(X_hist)

#load mode
clf_lbp = joblib.load('clf_lbp.model')
clf_hist = joblib.load('clf_hist.model')

eth_label ={'1':'apple','2':'car','3':'cow','4':'cup','5':'dog','6':'horse','7':'pear','8':'tomato'}
corel_label = {'0': 'window','1': 'bear','2': 'wolf','3': 'lion','4': 'elephant','5': 'tiger','6': 'snow moutain', '7': 'swimmer','8': 'Egypt','9': 'vegetabel',
               '10': 'people','11':'dog','12':'cloud','13':'mushroom','14':'castle','15':'pictorial','16':'African','17':'beverage','18':'fighter','19':'planet'
               ,'20':'poster'}
class Example(flx.PyComponent):

    def init(self):
        super().init()
        
        with ui.VSplit():
            #url = 'http://www.w3schools.com/tags/mov_bbb.mp4'
            #ui.VideoWidget(source=url)
            #ui.YoutubeWidget(source='RG1P8MQS1cU')
            with ui.GroupWidget(title = 'Please Input:'):
                with flx.HBox(flex=1):
                    self.dataset = flx.LineEdit(placeholder_text='Dataset name:ETH')
                    self.k_parameter = flx.LineEdit(placeholder_text='K: for KNN')
                    self.imgnum = flx.LineEdit(placeholder_text='Image number')
            
            with ui.GroupWidget(title = 'Dataset information:',flex = 8):
                #self.a = ui.ImageWidget( source = 'https://github.com/Ziyu-Z/6895/blob/raw/des1.png',flex=1,stretch=True)
                with ui.TabLayout() as self.t:
                    #with ui.Widget(title = 'Description'):
                    self.a = ui.ImageWidget(title = 'Description',source = 'https://github.com/Ziyu-Z/6895/raw/master//des1.png')
                    self.b = ui.ImageWidget(title = 'Data', source = 'https://github.com/Ziyu-Z/6895/raw/master//des2.png')
                    self.c = ui.ImageWidget(title = 'Accuracy', source = 'https://github.com/Ziyu-Z/6895/raw/master//accracy.png')
                    self.d = ui.ImageWidget(title = 'Stability', source = 'https://github.com/Ziyu-Z/6895/raw/master//sta.png')

                #with ui.VBox():
                 #   self.info = ui.ImageWidget(source = eth_des,flex=15,stretch=True)
                    
                #with ui.PyWidget():
                    #ui.IFrame(url= 'D:\\ColumbiaU\2001-2005\Research\ESGtaxonomy\TaxoRL\datasets\MAG\render.html')
                              #'https://github.com/Ziyu-Z/6895/blob/raw/render.html')
                              #
                              #'http://flexx.readthedocs.io')
            
            with ui.GroupWidget(title = 'Predicted result:',flex = 1):
                with ui.VBox():
                    self.result = ui.Label(text='Predicted result:')
                
            with flx.HBox(flex=10):
                with ui.VBox(flex=1):
                    #with flx.VBox(flex=1):
                    with ui.GroupWidget(title = 'Input Image:'):
                            #self.inimage = flx.Label(text='Input Image:',flex=1)
                            #url1 = imd
                            #stretch = ui.CheckBox(text='Stretch')
                            self.image1 = ui.ImageWidget(flex=15,stretch=True)#, stretch=lambda:stretch.checked
                with ui.VBox(flex=1):
                    #with flx.VBox(flex=1):
                    with ui.GroupWidget(title = 'Detect Object:'):
                            #self.outimage = flx.Label(text='Detect Object:',flex=1)
                            #url2 = self.get_path()
                            #'https://github.com/Ziyu-Z/6895/raw/master/ETH/3/cow1-022-090.png'
                            #stretch = ui.CheckBox(text='Stretch')
                            self.image2 = ui.ImageWidget(flex=15,stretch=True)#, stretch=lambda:stretch.checked
                    
    '''            
    @flx.reaction('imgnum.text')           
    def get_path2(self,*events):
        X,Y = self.import_datasets('eth')
        p = X[int(self.imgnum.text)]
        #p = '1/apple9-035-045.png'
        s = 'https://github.com/Ziyu-Z/6895/raw/master/ETH/'+str(p)
        self.image2.set_source(s)
    ''' 
    @flx.reaction('imgnum.text','k_parameter.text')           
    def get_path(self,*events):
        X,Y = self.import_datasets('eth')
        p = X[int(self.imgnum.text)]
        #p = '1/apple9-035-045.png'
        path = 'https://github.com/Ziyu-Z/6895/raw/master/ETH/'+str(p)
        s1,s2 = self.image_segmentation(path,int(self.k_parameter.text))
        self.image1.set_source(s1)
        self.image2.set_source(s2)
    
    def image_segmentation(self,path,k):
        i = io.imread(path)
        label_image = segmentation.slic(i,n_segments = k)
        image_label_overlay = color.label2rgb(label_image,image = i)
        
        fig1,ax0= plt.subplots(1,1, figsize=(8, 6))
        fig2,ax1= plt.subplots(1,1, figsize=(8, 6))
        #ax0.imshow(cleared,plt.cm.gray)
        ax0.imshow(i)
        ax1.imshow(image_label_overlay)

        MR1 = 0
        MR2 = 0
        MC1 = 0
        MC2 = 0
        A,B = i.shape[:2]
        dis = max(A,B)
        a = A/2
        b = B/2

        for region in measure.regionprops(label_image): #循环得到每一个连通区域属性集


            if region.area < 400:
                continue

            minr, minc, maxr, maxc = region.bbox
            d = (maxc - minc)/2 + minc
            c = (maxr - minr)/2 + minr
            D = self.distance(a,b,c,d)
            #rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
            #                              fill=False, edgecolor='blue', linewidth=2)
            #ax1.add_patch(rect)
            #if D < dis:
            if (abs(maxc - minc)* abs(maxr - minr)) > (abs(MC2-MC1)* abs(MR2-MR1)):
                if D < dis:
                    MR1 = minr
                    MR2 = maxr
                    MC1 = minc
                    MC2 = maxc
                    dis = D

        rect = mpatches.Rectangle((MC1, MR1), MC2 - MC1, MR2 - MR1,
                                  fill=False, edgecolor='red', linewidth=2)
        ax1.add_patch(rect)

        buffer = BytesIO()
        
        fig1.savefig(buffer)  
        plot_data1 = buffer.getvalue()
        imb1 = base64.b64encode(plot_data1)
        ims1 = imb1.decode()
        imd1 = "data:image/png;base64,"+ims1
        
        buffer = BytesIO()
        fig2.savefig(buffer)  
        plot_data2 = buffer.getvalue()
        imb2 = base64.b64encode(plot_data2)
        ims2 = imb2.decode()
        imd2 = "data:image/png;base64,"+ims2
        
        return imd1,imd2
    
    def distance(self,a,b,c,d):
        p1=np.array([a,b])
        p2=np.array([c,d])
        p3=p2-p1
        p4=math.hypot(p3[0],p3[1])

        return p4
        
        
        

    def import_datasets(self,name):
        
        X = [] #image paths
        Y = [] #image labels
        
        if name == 'eth':
            n = 8
            for i in range(1,n+1):
                for f in os.listdir('.\\photo\\%s' % i):
                    X.append(str(i) + '/' + str(f))
                    Y.append(i)
            else:
                for f in os.listdir('.\\Corel10k'):
                    if '.jpg' in str(f):
                        X.append(str(f))
                        p = f.find('_')
                        Y.append(f[:p])
            
        X = np.array(X)
        Y = np.array(Y) 
            
        return X,Y
    
    @flx.reaction('dataset.text', 'imgnum.text')
    def get_result_info(self, *events):
        r_hist = int(clf_hist.predict([X_hist[int(self.imgnum.text)]]))
        r_lbp = int(clf_lbp.predict([X_lbp[int(self.imgnum.text)]]))
        #actual = self.Y[int(self.imgnum.text)]
        self.result.set_text('prediction 1: '+eth_label[str(r_lbp)]+'\n'+'prediction 2:'+eth_label[str(r_hist)])
        
    @flx.reaction('dataset.text', 'imgnum.text')
    def get_image_info(self, *events):
        self.label.set_text('image/ ' + self.dataset.text + ' /' + self.imgnum.text+ '.png')

In [ ]:
app = flx.App(Example)
rt = launch('file://...','app')
#app.launch(runtime = rt) 
app.launch(runtime_kwargs = rt) 
rt.close()

In [ ]:
app = flx.App(Example)
app.Session()
app.export('example.html', link=0)  # Export to single file